In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 30768
db = "AAC" 
collection = "animals" 

# Connect to database via CRUD Module
shelter = AnimalShelter(db, collection, username, password, host, port)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Chris Wong | SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset Filters', 'value': 'reset'}
            ],
            value='WR'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="pie-chart",
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])



#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')]
)
def update_dashboard(value):
    df = pd.DataFrame(list(shelter.read({})))
    if value == 'WR':
        df = pd.DataFrame(list(shelter.read(
            {
                "breed": {"$in": ["Chesapeake Bay Retriever", "Labrador Retriever Mix", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$lte": 26.0}
            }
        )))
    elif value == 'MWR':
        df = pd.DataFrame(list(shelter.read(
            {
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", 
                                  "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
            }
        )))
    elif value == 'DIT':
        df = pd.DataFrame(list(shelter.read(
            {
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd", 
                                  "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
            }
        )))

    # Convert DataFrame to list of dictionaries
    data = df.to_dict('records')
    return data

    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # This callback function updates the style of selected columns in the data table.
    # It changes the background color of the selected columns to a light blue shade.
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Generating Pie Chart
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def figure_chart(data):
    # This callback function generates a pie chart based on data provided.
    # It checks if the 'breed' column exists in the data and creates the chart with breed data.
    dff = pd.DataFrame.from_dict(data)    
    if 'breed' in dff.columns:
        fig = px.pie(
            dff,
            names='breed',
        )
        return fig
    else:
        # If 'breed' column does not exist, it returns an empty dictionary to display no chart.
        return {}





@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, derived_virtual_selected_rows):
    
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    animalChoice = None
    if not derived_virtual_selected_rows:
        animalChoice = dff.iloc[0]
    else:
        animalChoice = dff.iloc[derived_virtual_selected_rows[0]]
        
    latitude = animalChoice[13]
    longitude = animalChoice[14]
    breed = animalChoice[4]
    name = animalChoice[9]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   dl.Marker(position=[latitude, longitude], children=[
                       dl.Tooltip(breed),
                       dl.Popup([
                           html.H1("Animal Name"),
                           html.P(name)
                       ])
                   ])
               ])
    ]




app.run_server(debug=True)

Dash app running on http://127.0.0.1:10489/
